## Work
1. 請比較使用 l1, l1_l2 及不同比例下的訓練結果

In [1]:
import os
import tensorflow.keras as keras
from tensorflow.keras.regularizers import l1, l2, l1_l2
import itertools

In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:

def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128],regularizer=l1(1e-5)):
    input_layer = keras.layers.Input(input_shape)
    for i, n_units in enumerate(num_neurons):
        if i == 0:              
            x = keras.layers.Dense(units=n_units,
                                    activation='relu',
                                    name="hidden_layer"+str(i+1),
                                    kernel_regularizer=regularizer)(input_layer)
        else:
            x = keras.layers.Dense(units=u_units,
                                    activation='relu',
                                    name="hidden_layer"+str(i+1),
                                    kernel_regularizer=regularizer)(x)
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    
    return model

In [6]:
"""Code Here
設定超參數
"""
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 256
MOMENTUM = 0.95
l1_ratio = 1e-5
l2_ratio = 1e-5
REGULARIZER = [l1(l1_ratio), l2(l2_ratio), l1_l2(l1_ratio, l2_ratio)]

In [7]:
results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
lr = LEARNING_RATE
momentum = MOMENTUM
for regularizer in REGULARIZER:
    keras.backend.clear_session()
    print("Experiment with LR = %.6f, Momentum = %.2f, Regularizer = %s" % (lr, momentum, str(regularizer)))
    model = build_mlp(input_shape=x_train.shape[1:], regularizer=REGULARIZER)
    model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)
    model.fit(x_train, y_train,
              epochs = EPOCHS,
              batch_size = BATCH_SIZE,
              validation_data=(x_test, y_test), 
              shuffle=True)
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["accuracy"]
    valid_acc = model.history.history["val_accuracy"]
    
    exp_name_tag = "exp-%s" % str(regularizer)
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}

Experiment with LR = 0.001000, Momentum = 0.95, Regularizer = <tensorflow.python.keras.regularizers.L1L2 object at 0x0000017004BA6BE0>


ValueError: ('Could not interpret regularizer identifier:', [<tensorflow.python.keras.regularizers.L1L2 object at 0x0000017004BA6BE0>, <tensorflow.python.keras.regularizers.L1L2 object at 0x0000017004BA6F28>, <tensorflow.python.keras.regularizers.L1L2 object at 0x0000017004BA64E0>])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
color_bar = ["r", "g", "b", "y", "m", "k"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.ylim([0, 5])
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()